In [4]:
import pandas

COLS_TO_USE = [4,5,6,9,10,12,13,14,15,16]
HEADER_NAMES = ['size','weight','recieved_time','fee','confirmed_block_height','confirm_time','waiting_time','fee_rate','enter_block_height','no_block_confirm']

d1 = pandas.read_csv('TimetxinBlock621500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d2 = pandas.read_csv('TimetxinBlock622000.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d3 = pandas.read_csv('TimetxinBlock622500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
dataFrame = pandas.concat([d1, d2, d3])

transaction_count = len(dataFrame)

dataFrame.head(20)

,size,weight,recieved_time,fee,confirmed_block_height,confirm_time,waiting_time,fee_rate,enter_block_height,no_block_confirm
0,223,892,1583793568,2486,621001,1583794214,646,11.147982,621000,2
1,223,892,1583791819,2486,621001,1583794214,2395,11.147982,620998,4
2,223,892,1583793885,2486,621001,1583794214,329,11.147982,621001,1
3,223,892,1583793700,2486,621001,1583794214,514,11.147982,621000,2
4,223,892,1583791354,2486,621001,1583794214,2860,11.147982,620998,4
5,223,892,1583791731,2486,621001,1583794214,2483,11.147982,620998,4
6,371,1484,1583792024,4136,621001,1583794214,2190,11.148248,620998,4
7,371,1484,1583793603,4136,621001,1583794214,611,11.148248,621000,2
8,371,1484,1583793883,4136,621001,1583794214,331,11.148248,621001,1
9,371,1484,1583791921,4136,621001,1583794214,2293,11.148248,620998,4


In [5]:
P_GROUP_BOUNDS = [1,3,10]
block_confirm_max = dataFrame['no_block_confirm'].max()

df_p1 = dataFrame[dataFrame['no_block_confirm'].between(0, P_GROUP_BOUNDS[0])]
df_p2 = dataFrame[dataFrame['no_block_confirm'].between((P_GROUP_BOUNDS[0]+1), P_GROUP_BOUNDS[1])]
df_p3 = dataFrame[dataFrame['no_block_confirm'].between((P_GROUP_BOUNDS[1]+1), P_GROUP_BOUNDS[2])]
df_p4 = dataFrame[dataFrame['no_block_confirm'].between((P_GROUP_BOUNDS[2]+1), block_confirm_max)]

transaction_count_p1 = len(df_p1)
transaction_count_p2 = len(df_p2)
transaction_count_p3 = len(df_p3)
transaction_count_p4 = len(df_p4)

lambda_p1 = float(transaction_count_p1) / (float(df_p1['recieved_time'].max()) - float(df_p1['recieved_time'].min()))
print("lambda_p1: ", lambda_p1)
lambda_p2 = float(transaction_count_p2) / (float(df_p2['recieved_time'].max()) - float(df_p2['recieved_time'].min()))
print("lambda_p2: ", lambda_p2)
lambda_p3 = float(transaction_count_p3) / (float(df_p3['recieved_time'].max()) - float(df_p3['recieved_time'].min()))
print("lambda_p3: ", lambda_p3)
lambda_p4 = float(transaction_count_p4) / (float(df_p4['recieved_time'].max()) - float(df_p4['recieved_time'].min()))
print("lambda_p4: ", lambda_p4)

service_time = 600
mu = 1/service_time
print("Mu: ", mu)

block_groups = dataFrame.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))
print("Mean Block Size: ", mean_block_size)

lambda_p1:  1.9164135440894505
lambda_p2:  0.5319602867074807
lambda_p3:  0.36553531432826897
lambda_p4:  0.3759231995900763
Mu:  0.0016666666666666668
Mean Block Size:  2266.0


In [8]:
current_lambda = 0.0

def NewtonMethod(lam, m_u, block_size, x0, epsilon, max_iteration):

    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam

    xn = x0
    for n in range(0, max_iteration):
        fxn = fx(xn)
        if abs(fxn) < epsilon:
            return xn
        
        dfxn = dfx(xn)
        if dfxn == 0:
            return None

        xn = xn - fxn/dfxn

    return None

z1 = NewtonMethod(lambda_p1, mu, mean_block_size, 0, 1e-10, 500)
L1 = z1 / (1 - z1)
W1 = L1 / lambda_p1
print("z1: ", z1)
print("L1: ", L1)
print("Wait Times P1: ", W1)

z2 = NewtonMethod((lambda_p1+lambda_p2), mu, mean_block_size, 0, 1e-10, 500)
L2 = (z2 / (1 - z2)) - L1
W2 = L2 / lambda_p2
print("z2: ", z2)
print("L2: ", L2)
print("Wait Times P2: ", W2)

z3 = NewtonMethod((lambda_p1+lambda_p2+lambda_p3), mu, mean_block_size, 0, 1e-10, 500)
L3 = (z3 / (1 - z3)) - L2
W3 = L3 / lambda_p3
print("z3: ", z3)
print("L3: ", L3)
print("Wait Times P3: ", W3)

z4 = NewtonMethod((lambda_p1+lambda_p2+lambda_p3+lambda_p4), mu, mean_block_size, 0, 1e-10, 500)
L4 = (z4 / (1 - z4)) - L3
W4 = L4 / lambda_p4
print("z4: ", z4)
print("L4: ", L4)
print("Wait Times P4: ", W4)

z1:  0.9993147806098512
L1:  1458.3866057743926
Wait Times P1:  760.9978599203225
z2:  0.9995855087963951
L2:  953.2098288489833
Wait Times P2:  1791.881560837535
z3:  0.9997263332086644
L3:  2699.8688963034715
Wait Times P3:  7386.068569776693
z4:  0.9998465892552025
L4:  3817.579347463318
Wait Times P4:  10155.210829302845


In [9]:
from scipy.optimize import newton

def f(x):
    return lambda_p1*(1-x) - mu*x*(1-x**mean_block_size)

prime = lambda x: mu*(mean_block_size*x**mean_block_size + x**mean_block_size - 1) - lambda_p1

x=0
x0 = newton(f, x, fprime=None, args=(), tol=1.0e-8, maxiter=50, fprime2=None)
print('z1: ', x0)

z1:  0.9993147806470818
